# New Section

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


In [21]:
torch.manual_seed(42)

train_dir = './PetImages'
validation_dir = './PetImages'

IMG_SIZE = (64,64)

transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])

class SafeImageFolder(ImageFolder):
  def __getitem__(self, index):
    try:
      return super(SafeImageFolder, self).__getitem__(index)
    except UnidentifiedImageError:
      print(f"Skipping this image: {self.samples[index][0]}")
      return None

def safe_collate(batch):
  batch = [item for item in batch if item is not None]
  if len(batch) == 0:
    return None
  return torch.utils.data.dataloader.default_collate(batch)




In [22]:
train_dataset = SafeImageFolder(train_dir, transform=transform)
validation_dataset = SafeImageFolder(validation_dir, transform=transform)
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=safe_collate)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=safe_collate)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './PetImages'

In [ ]:
class CatsDogsMLP(nn.Module):
  def __init__(self):
    super(CatsDogsMLP, self).__init__()
    self.flatten = nn.Flatten() # 64x64x1 to 4096
    self.layer1 = nn.Linear(64*64*1, 512)
    self.relu1 = nn.ReLU()
    self.layer2 = nn.Linear(512,256)
    self.relu2 = nn.ReLU()
    self.layer3 = nn.Linear(256,128)
    self.relu3 = nn.ReLU()
    self.output = nn.Linear(128,1)
    self.sigmoid = nn.Sigmoid()
  def forward(self,x):
    x = self.flatten(x)
    x = self.layer1(x)
    x = self.relu1(x)
    x = self.layer2(x)
    x = self.relu2(x)
    x = self.layer3(x)
    x = self.relu3(x)
    x = self.output(x)
    x = self.sigmoid(x)
    return x

In [ ]:
model = CatsDogsMLP()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
NUM_EPOCHS = 10
train_losses = []
val_losses = []
for epoch in range(NUM_EPOCHS):
  model.train()
  running_loss = 0.0
  train_bar = tqdm(train_loader, desc=f'epoch {epoch+1}/{NUM_EPOCHS}[Train]')
  for batch in train_bar:
    if batch is None:
      continue
    images, labels = batch
    images,labels = images, labels.float().view(-1,1)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    train_bar.set_postfix({'loss': loss.item()})

  avg_train_loss = running_loss/ len([b for b in train_loader if b is not None])
  train_losses.append(avg_train_loss)

  model.eval()
  val_loss = 0.0
  correct = 0
  total = 0
  val_bar = tqdm(validation_loader, desc=f'epoch {epoch+1}/{NUM_EPOCHS}[Val]')
  with torch.no_grad():
    for batch in val_bar:
      if batch is None:
        continue
    images, labels = batch
    images,labels = images, labels.float().view(-1,1)
    outputs = model(images)
    val_loss += criterion(outputs, labels).item()
    predicted = (outputs >=0.5).float()
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    val_bar.set_postfix({'val_loss': val_loss / (total / BATCH_SIZE)})
  avg_val_loss = val_loss / len([b for b in validation_loader if b is not None])
  val_losses.append(avg_val_loss)
  print(f'Epoch [{epoch+1}/{NUM_EPOCHS}],'
        f'Train Loss: {avg_train_loss:.4f}',
        f'Val Loss: {avg_val_loss:.4f}',
        f'Val Accusracy: {100 * correct/total:.2f}%')

epoch 1/10[Train]:  19%|█▉        | 147/782 [02:22<10:15,  1.03it/s, loss=0.675]


NameError: name 'UnidentifiedImageError' is not defined

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(range(1, NUM_EPOCHS+1), train_losses, label='Training Loss', marker='o')
plt.plot(range(1, NUM_EPOCHS+1), val_losses, label='Validation Loss', marker='o')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.savefig('loss_plot.png')

In [ ]:
torch.save(model.state_dict(), 'cats_dogs_mlp.pth')